# Tutorial: Create an adder simulator using python Bonsai SDK 

Adder is a very simple dummy simulator which just adds 2 numbers and validates if Bonsai platform returned the sum of those 2 numbers or not.


### Step1: Import required classes and modules.

In [1]:
import os
import sys
import time
from datetime import datetime
import requests
import json
from bonsai.simulatorapi.simulator_api import *
from bonsai.simulatorapi.models import *
from bonsai.simulatorapi import *
from bonsaiclient import BonsaiClient, CreateSimContext

### Step2: specify the required parameters or read from env vars.
1. timeout: Bonsai protocol needs the simulator to keep pinging constantly, while it knows that simulating the events can take time and depends on simulator type. So this param will give bonsai an idea about latencies in simulator and when to delete a simulator session automatically if sim didn't respond within this time.
2. api-host: Root URL for bonsai server. It's https://api.bons.ai
3. workspace: Name of your Bonsai workspace.
4. access-key: AccesKey to authenticate calls from this simulator. you can generate access-keys using Bonsai platform.
5. sim-context: This is the most important part of registration information for a new session. 
   It has the purpose, which contains, brainName, brainVersion and concept and this session      will only be used to train those active brainversions, hacing the same conceptname.


In [2]:
rest_api = None
session_id = ''

# parse the arguments and environment variables
# you can sue this helper function to create sim-context string as well. 
# simcontext = CreateSimContext("Train", workspace, brainName, brainVersion, conceptName)

params = {
    'timeout': 0.0,
    'api-host': os.getenv('SIM_API_HOST', 'https://api.bons.ai'),
    'workspace': os.getenv('SIM_WORKSPACE', ''),
    'access-key': os.getenv('SIM_ACCESS_KEY', 'bonsai nothing'),
    'sim-context': os.getenv('SIM_CONTEXT', ''),
}

if not params['workspace']:
    raise RuntimeError('Must define a workspace in SIM_WORKSPACE or --workspace')

#print(f'using params {params}')

# build the api handler
rest_api = BonsaiClient(
    workspace=params['workspace'],
    host=params['api-host'],
    access_key=params['access-key']
    )

### Step3: Now Let's create a new simulator session with bonsai platform.

In [3]:

register_response: SimulatorSessionResponse = rest_api.create_session(
    'the_simulator',
    timeout_seconds=params['timeout'],
    simulator_context=params['sim-context'],
)

print('Registered session: -> {}'.format(register_response.session_id))

Creating session with {
   "capabilities": {},
   "description": {},
   "name": "the_simulator",
   "simulatorContext": "{\"deploymentMode\": \"Testing\", \"simulatorClientId\": \"123456\", \"purpose\": { \"action\": \"Train\", \"target\": { \"workspaceName\": \"00a5e6686af84f1f\", \"brainName\": \"AdderSdk3\", \"brainVersion\": 4, \"conceptName\": \"addition\" } } }",
   "timeout": 0.0
} 


Registered session: -> 901025167_10.244.53.180


### Step4: Initial setup for State <--> Action Loop.
Let's define intial state of our simulator and reset the stats.

Note: sequence_id is an important part of this workflow, it's part of SDK3 format for bonsai platform. It starts with 1 and Bonsai platform update this as you call advance and return in each advance response. It's simulators responsibility to return the same sequence_id everytime and not change it in order to conform to SDK3 protocol.


In [4]:
# extract the session id from the response
session_id = register_response.session_id
sequence_id = 1

# set up our initial state
value1 = 2
value2 = 5
reward = 0.0
total_episode_reward = 0.0
episode_iterations = 0

# Loop until Unregister or Ctrl+C
event_type = 'Idle'

### Step5:  event loop or state<-->action loop.
Loop, where simulator will send state to Bonsai platform and will receive back Events. Bonsai platform, advance call will return one of these events.

1. EpisodeStart: signifies that platform is starting a new episode. it's good time to reset some episode bound stats and reset the states of simulation.
2. EpisodeStep: Bonsai platform sent an RL action and you need to advance simulation by simulating the sent action.
3. EpisodeFinish: Current episode is finished. 
4. Unregister: platform is asking you to unregister/delete the simulator session. you can recreate session, if you stil want to train further your brain.
5. Idle: this session did not get used to train any brain so it sent idle. in the idle response there maybe some callback time, if present then try after that much time, else retry sending requests.

In [5]:
try:
    while event_type != 'Unregister':

        # invoke Advance
        advance_time = datetime.utcnow()
        advance_response: Event = rest_api.advance(
            session_id, sequence_id,
            {'value1': value1, 'value2': value2, '_reward': reward})

        if 0 == (episode_iterations % 100):
            print(advance_time,'ADVANCE -> {}'.format(advance_response))

        event: Event = advance_response
        event_type = event.type

        # Handle the response
        if event_type == 'Idle':
            waitForSeconds = event.idle.callback_time
            if waitForSeconds > 0.0:
                print('Sleeping for', waitForSeconds, 'seconds')
                time.sleep(waitForSeconds)
            # just try again

        elif event_type == 'Unregister':
            # will break out of loop! we will delete session later.
            rest_api = None
            session_id = ''

        elif event_type == 'EpisodeStart':
            # re-initialize the simulator state
            value1 = 2
            value2 = 5
            reward = 0.0
            total_episode_reward = 0
            episode_iterations = 0

        elif event_type == 'EpisodeStep':
            # check the sum in the response
            action = event.episode_step.action
            if action['sum'] == (value1 + value2)%10:
                reward = 1.0
            else:
                reward = 0.0

            total_episode_reward += reward
            episode_iterations += 1

            # update the simulator state
            value1 += 2
            if value1 > 9:
                value1 -= 10

            value2 += 3
            if value2 > 9:
                value2 -= 10

        elif event_type == 'EpisodeFinish':
            # just print out the episode finish
            print('Episode finished! Total reward = {} / {}'.format(total_episode_reward, episode_iterations))

        # the simulator's sequence id is always the last one it received
        sequence_id = event.sequence_id
except KeyboardInterrupt:
        pass
finally:
    if rest_api is not None and session_id:
        print('\n\n\nUnregistering {}'.format(session_id))
        deregister_response = rest_api.delete_session(session_id)
        print(datetime.utcnow(), 'UNREGISTER ->', deregister_response)

2020-05-21 01:39:55.634226 ADVANCE -> {'additional_properties': {}, 'type': <EventTypesEventType.episode_start: 'EpisodeStart'>, 'session_id': '901025167_10.244.53.180', 'sequence_id': 2, 'episode_start': <bonsai.simulatorapi.models.episode_start_py3.EpisodeStart object at 0x105c7f750>, 'episode_step': None, 'episode_finish': None, 'playback_start': None, 'playback_step': None, 'playback_finish': None, 'idle': None, 'registered': None, 'unregister': None, 'kind_case': None}
2020-05-21 01:40:00.108775 ADVANCE -> {'additional_properties': {}, 'type': <EventTypesEventType.episode_step: 'EpisodeStep'>, 'session_id': '901025167_10.244.53.180', 'sequence_id': 3, 'episode_start': None, 'episode_step': <bonsai.simulatorapi.models.step_py3.Step object at 0x105c84190>, 'episode_finish': None, 'playback_start': None, 'playback_step': None, 'playback_finish': None, 'idle': None, 'registered': None, 'unregister': None, 'kind_case': None}



Unregistering 901025167_10.244.53.180
Successfully deleted